In [1]:
from time import sleep
import time
from playsound import playsound
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta
import multiprocessing as mp
from queue import Empty
import warnings

import matplotlib.pyplot as plt

from portilooplot.jupyter_plot import ProgressPlot
from frontend import Frontend
from leds import LEDs, Color

from IPython.display import clear_output

In [2]:
FREQUENCY = 250
SAMPLE_TIME = 1 / FREQUENCY

DEFAULT_FRONTEND_CONFIG = [
    # nomenclature: name [default setting] [bits 7-0] : description
    # Read only ID:
    0x3E, # ID [xx] [REV_ID[2:0], 1, DEV_ID[1:0], NU_CH[1:0]] : (RO)
    # Global Settings Across Channels:
    0x96, # CONFIG1 [96] [1, DAISY_EN(bar), CLK_EN, 1, 0, DR[2:0]] : Datarate = 250 SPS
    0xC0, # CONFIG2 [C0] [1, 1, 0, INT_CAL, 0, CAL_AMP0, CAL_FREQ[1:0]] : No tests
    0x60, # CONFIG3 [60] [PD_REFBUF(bar), 1, 1, BIAS_MEAS, BIASREF_INT, PD_BIAS(bar), BIAS_LOFF_SENS, BIAS_STAT] : Power-down reference buffer, no bias
    0x00, # LOFF [00] [COMP_TH[2:0], 0, ILEAD_OFF[1:0], FLEAD_OFF[1:0]] : No lead-off
    # Channel-Specific Settings:
    0x61, # CH1SET [61] [PD1, GAIN1[2:0], SRB2, MUX1[2:0]] : Channel 1 active, 24 gain, no SRB2 & input shorted
    0x61, # CH2SET [61] [PD2, GAIN2[2:0], SRB2, MUX2[2:0]] : Channel 2 active, 24 gain, no SRB2 & input shorted
    0x61, # CH3SET [61] [PD3, GAIN3[2:0], SRB2, MUX3[2:0]] : Channel 3 active, 24 gain, no SRB2 & input shorted
    0x61, # CH4SET [61] [PD4, GAIN4[2:0], SRB2, MUX4[2:0]] : Channel 4 active, 24 gain, no SRB2 & input shorted
    0x61, # CH5SET [61] [PD5, GAIN5[2:0], SRB2, MUX5[2:0]] : Channel 5 active, 24 gain, no SRB2 & input shorted
    0x61, # CH6SET [61] [PD6, GAIN6[2:0], SRB2, MUX6[2:0]] : Channel 6 active, 24 gain, no SRB2 & input shorted
    0x61, # CH7SET [61] [PD7, GAIN7[2:0], SRB2, MUX7[2:0]] : Channel 7 active, 24 gain, no SRB2 & input shorted
    0x61, # CH8SET [61] [PD8, GAIN8[2:0], SRB2, MUX8[2:0]] : Channel 8 active, 24 gain, no SRB2 & input shorted
    0x00, # BIAS_SENSP [00] [BIASP8, BIASP7, BIASP6, BIASP5, BIASP4, BIASP3, BIASP2, BIASP1] : No bias
    0x00, # BIAS_SENSN [00] [BIASN8, BIASN7, BIASN6, BIASN5, BIASN4, BIASN3, BIASN2, BIASN1] No bias
    0x00, # LOFF_SENSP [00] [LOFFP8, LOFFP7, LOFFP6, LOFFP5, LOFFP4, LOFFP3, LOFFP2, LOFFP1] : No lead-off
    0x00, # LOFF_SENSN [00] [LOFFM8, LOFFM7, LOFFM6, LOFFM5, LOFFM4, LOFFM3, LOFFM2, LOFFM1] : No lead-off
    0x00, # LOFF_FLIP [00] [LOFF_FLIP8, LOFF_FLIP7, LOFF_FLIP6, LOFF_FLIP5, LOFF_FLIP4, LOFF_FLIP3, LOFF_FLIP2, LOFF_FLIP1] : No lead-off flip
    # Lead-Off Status Registers (Read-Only Registers):
    0x00, # LOFF_STATP [00] [IN8P_OFF, IN7P_OFF, IN6P_OFF, IN5P_OFF, IN4P_OFF, IN3P_OFF, IN2P_OFF, IN1P_OFF] : Lead-off positive status (RO)
    0x00, # LOFF_STATN [00] [IN8M_OFF, IN7M_OFF, IN6M_OFF, IN5M_OFF, IN4M_OFF, IN3M_OFF, IN2M_OFF, IN1M_OFF] : Laed-off negative status (RO)
    # GPIO and OTHER Registers:
    0x0F, # GPIO [0F] [GPIOD[4:1], GPIOC[4:1]] : All GPIOs as inputs
    0x00, # MISC1 [00] [0, 0, SRB1, 0, 0, 0, 0, 0] : Disable SRBM
    0x00, # MISC2 [00] [00] : Unused
    0x00, # CONFIG4 [00] [0, 0, 0, 0, SINGLE_SHOT, 0, PD_LOFF_COMP(bar), 0] : Single-shot, lead-off comparator disabled
]

FRONTEND_CONFIG = [
    0x3E, # ID (RO)
    0x94, # CONFIG1 [95] [1, DAISY_EN(bar), CLK_EN, 1, 0, DR[2:0]] : Datarate = 500 SPS
    0xD0, # CONFIG2 [C0] [1, 1, 0, INT_CAL, 0, CAL_AMP0, CAL_FREQ[1:0]]
    0xE0, # CONFIG3 [E0] [PD_REFBUF(bar), 1, 1, BIAS_MEAS, BIASREF_INT, PD_BIAS(bar), BIAS_LOFF_SENS, BIAS_STAT] : Power-down reference buffer, no bias
    0x00, # No lead-off
    0x03, # CH1SET [60] [PD1, GAIN1[2:0], SRB2, MUX1[2:0]]
    0x00, # CH2SET
    0x00, # CH3SET
    0x00, # CH4SET
    0x00, # CH5SET voltage
    0x00, # CH6SET voltage
    0x00, # CH7SET test
    0x04, # CH8SET temperature
    0x00, # BIAS_SENSP
    0x00, # BIAS_SENSN
    0xFF, # LOFF_SENSP Lead-off on all positive pins?
    0xFF, # LOFF_SENSN Lead-off on all negative pins?
    0x00, # Normal lead-off
    0x00, # Lead-off positive status (RO)
    0x00, # Lead-off negative status (RO)
    0x00, # All GPIOs as output ?
    0x20, # Enable SRB1
]

def filter_24(value):
    return (value * 4.5) / (2**23 - 1)  # 23 because 1 bit is lost for sign

def filter_2scomplement(value):
    if (value & (1 << 23)) != 0:
        value = value - (1 << 24)
    return filter_24(value)

def filter_2scomplement_np(value):
    v = np.where((value & (1 << 23)) != 0, value - (1 << 24), value)
    return filter_24(v)


class LiveDisplay():
    def __init__(self, datapoint_dim=8, window_len=100):
        self.datapoint_dim = datapoint_dim
        # self.queue = [mp.Queue()] * datapoint_dim
        self.queue = mp.Queue()
        channel_names = [f"channel#{i+1}" for i in range(datapoint_dim)]
        channel_names[0] = "voltage"
        channel_names[7] = "temperature"
        self.pp = ProgressPlot(plot_names=channel_names, max_window_len=window_len)

    def add_datapoints(self, datapoints):
        """
        Adds 8 lists of datapoints to the plot
        
        Args:
            datapoints: list of 8 lists of floats (or list of 8 floats)
        """
        disp_list = []
        for datapoint in datapoints:
            d = [[elt] for elt in datapoint]
            disp_list.append(d)
        self.pp.update_with_datapoints(disp_list)
    
    def add_datapoint(self, datapoint):
        disp_list = [[elt] for elt in datapoint]
        self.pp.update(disp_list)

In [3]:
def _capture_process(q_data, q_out, q_in, duration, sample_time):
    """
    Args:
        q_data: multiprocessing.Queue: captured datapoints are put in the queue
        q_out: mutliprocessing.Queue: to pass messages to the parent process
            'STOP': end of the the process
        q_in: mutliprocessing.Queue: to pass messages from the parent process
            'STOP': stops the process
    """
    
    print(os.getpid())
    
    frontend = Frontend()
    leds = LEDs()
    leds.led2(Color.PURPLE)
    leds.aquisition(True)
    
    print(f"sample time: {sample_time}")
    
    try:
        data = frontend.read_regs(0x00, 1)
        assert data == [0x3E], "Wrong output"
        print("EEG Frontend responsive")
        leds.led2(Color.BLUE)

        print("Configuring EEG Frontend")
        frontend.write_regs(0x00, FRONTEND_CONFIG)
        data = frontend.read_regs(0x00, len(FRONTEND_CONFIG))
        assert data == FRONTEND_CONFIG, f"Wrong config: {data} vs {FRONTEND_CONFIG}"
        frontend.start()
        print("EEG Frontend configured")
        leds.led2(Color.PURPLE)
        while not frontend.is_ready():
            pass
        print("Ready for data")

        # Set up of leds
        leds.aquisition(True)
        sleep(0.5)
        leds.aquisition(False)
        sleep(0.5)
        leds.aquisition(True)

        c = True

        it = 0
        t_start = time.time()
        t_max = t_start + duration
        t = t_start
        
        # first sample:
        reading = frontend.read()
        datapoint = reading.channels()
        q_data.put(datapoint)
        
        t_next = t + sample_time
        
        # sampling loop:
        while c and t < t_max:
            # sleep until next time-step
            t = time.time()
            if t <= t_next:
                time.sleep(t_next - t)
            else:
                warnings.warn(f"Timeout: {t - t_next}s overflow, size queue data: {q_data.qsize()}")
                
            t_next += sample_time


            # sample
            # reading = frontend.wait_new_data()
            reading = frontend.read()
            datapoint = reading.channels()
            q_data.put(datapoint)
            

            # Check for messages  # this takes too long :/
#             try:
#                 message = q_in.get_nowait()
#                 if message == 'STOP':
#                     c = False
#             except Empty:
#                 pass
            it += 1
        
        tot = (t - t_start) / it        

        print(f"Average frequency: {1 / tot} Hz for {it} samples")

        leds.aquisition(False)

    finally:
        frontend.close()
        leds.close()
        q_in.close()
        q_out.put('STOP')

In [4]:
q_messages_send = mp.Queue()
q_messages_recv = mp.Queue()
q_data = mp.Queue()

duration = 60

print(os.getpid())

# _capture_process(q_data, q_messages_recv, q_messages_send, duration, SAMPLE_TIME)

p = mp.Process(target=_capture_process, args=(q_data, q_messages_recv, q_messages_send, duration, SAMPLE_TIME))
p.start()

live_disp = LiveDisplay(window_len=500)

cc = True

while cc:
    try:
        mess = q_messages_recv.get_nowait()
        if mess == 'STOP':
            cc = False
    except Empty:
        pass

    # retrieve all data points from q_data and put them in a list of np.array:
    res = []
    c = True
    while c and len(res) < 25:
        try:
            point = q_data.get(timeout = 0.1)
            res.append(point)
        except Empty:
            c = False
    if len(res) == 0:
        time.sleep(SAMPLE_TIME)
        continue
    n_array = np.array(res)
    n_array = filter_2scomplement_np(n_array)
    to_add = n_array.tolist()
    live_disp.add_datapoints(to_add)

c = True 
while c:
    try:
        _ = q_data.get_nowait()
    except Empty:
        c = False

q_messages_recv.close()
q_data.close()

# p.join()

5060
5074


<IPython.core.display.Javascript object>

sample time: 0.004
EEG Frontend responsive
Configuring EEG Frontend
EEG Frontend configured
Ready for data


/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.0006971359252929688s overflow, size queue data: 14
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.0030024051666259766s overflow, size queue data: 15
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.0026307106018066406s overflow, size queue data: 16
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.002470731735229492s overflow, size queue data: 17
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.004503011703491211s overflow, size queue data: 18
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.00672602653503418s overflow, size queue data: 19
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.009312391281

/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.0807347297668457s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.08110928535461426s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.08287191390991211s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.08497142791748047s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.08661794662475586s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.0881657600402832s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 0.08860063552856445s overflow,

...

/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 12.901916027069092s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 12.903692960739136s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 12.904109716415405s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 12.906394720077515s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 12.908312559127808s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 12.908298969268799s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 12.910399675369263s overflow, size

/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.002161741256714s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.002682209014893s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.004991054534912s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.011358976364136s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.013856649398804s overflow, size queue data: 2
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.015066862106323s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.015699863433838s overflow, size

/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.085835456848145s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.088138580322266s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.088566541671753s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.090171098709106s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.092722654342651s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.09355640411377s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.095419645309448s overflow, size 

/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.190404891967773s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.190492391586304s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.192569732666016s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.194705963134766s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.19543743133545s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.195837259292603s overflow, size queue data: 1
/home/mendel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: Timeout: 13.197960615158081s overflow, size 

Average frequency: 194.870096693703 Hz for 11693 samples


In [5]:
t = [[2.4939729564157673, 0.04160160322208443, 0.03798437571339318, 0.03657031495217263, -0.05362594766926142, 0.044207101369750666, 0.04443562560506172, 0.15757121534004395], [2.494203089976679, 0.031165663142879385, 0.02662521918120613, 0.025235298303997313, -0.0648976641771393, 0.03350133103148115, 0.03366655512649478, 0.15757067889817702]]

In [6]:
t

[[2.4939729564157673,
  0.04160160322208443,
  0.03798437571339318,
  0.03657031495217263,
  -0.05362594766926142,
  0.044207101369750666,
  0.04443562560506172,
  0.15757121534004395],
 [2.494203089976679,
  0.031165663142879385,
  0.02662521918120613,
  0.025235298303997313,
  -0.0648976641771393,
  0.03350133103148115,
  0.03366655512649478,
  0.15757067889817702]]

In [7]:
[list(elt) for elt in zip(*t)]

[[2.4939729564157673, 2.494203089976679],
 [0.04160160322208443, 0.031165663142879385],
 [0.03798437571339318, 0.02662521918120613],
 [0.03657031495217263, 0.025235298303997313],
 [-0.05362594766926142, -0.0648976641771393],
 [0.044207101369750666, 0.03350133103148115],
 [0.04443562560506172, 0.03366655512649478],
 [0.15757121534004395, 0.15757067889817702]]